In [1]:
import pandas as pd
import numpy as np
from os.path import join
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix 
from sklearn import linear_model
from sklearn.svm import SVC

In [2]:
#Chargement des données
path_raw = '/Users/hermann-kevin/Desktop/PFR/data/raw_data/'
file_train = 'train.csv'

In [3]:
train = pd.read_csv(join(path_raw, file_train), error_bad_lines=False)

In [4]:
train.shape

(7377418, 6)

In [5]:
#On prend 300000 chansons dans la base 
trainPart = train.head(300000)

In [6]:
#Nombre d'utilisateurs
len(trainPart.msno.unique())

11330

In [7]:
# Les utilisateurs qui ont plus de 10 chansons dans le trainPart
usr_more_song = trainPart.groupby(['msno'])['song_id'].agg('count')
usr_more_song = pd.DataFrame(usr_more_song).reset_index()
usr_more_song.columns = [['msno', 'song_number']]
usr_most = usr_more_song[usr_more_song.song_number >= 10]
print("Le nombre d'utilisateur avec plus de 10 chansons: ", len(usr_most))

Le nombre d'utilisateur avec plus de 10 chansons:  7678


In [8]:
# Je donne un numero à chaque utilisateur. Pour avoir une variable catégorielle dans mes données par la suite
usr_id = usr_most.reset_index(drop=True)
usr_id = usr_id.reset_index()
usr_id = usr_id[['index', 'msno']]
usr_id.columns = ['index_mem', 'msno']
usr_id.head()

,index_mem,msno
0,0,+/SKX44s4ryWQzYzuV7ZKMXqIKQMN1cPz3M8CJ8CFKU=
1,1,+0+XaewkS3za58vHwKCVGwRYRvjig4lTtKDV/tOkzbU=
2,2,+0B4aHABar5ltaWDG4M6KCzNxpD6wnsvZpvh+o9KgtE=
3,3,+0U8n45rrg6b5WCy3wpqOUYCwZyLqecr3lux9K6ahMk=
4,4,+0e12C+p9dzDbOvKjt8eElKH9yZPshAstxjm60XFgSM=


In [9]:
# Je donne également un numero à chaque chanson
song_more = trainPart.groupby(['song_id'])['msno'] .agg('count')
song_more = pd.DataFrame(song_more).reset_index()

song_id = song_more.reset_index(drop=True)
song_id = song_id.reset_index()
song_id = song_id[['index', 'song_id']]
song_id.columns = ['index_song', 'song_id']
song_id.head()

,index_song,song_id
0,0,++7GdTgp8zbQLYOki7hVPEOHpu+KLZClsGrGiEuL2uI=
1,1,++CfKs1t1wU1t0q0UxCdRqGoDpToqgMPmYytklaqo9o=
2,2,++HlV9O/nnF8fmlWOgsrJSxO+rx75nHWuE6O3ykpVS8=
3,3,++NP2gO99EW6w6sm4E1XP20nx8FhbBxsK9fOvbPHwd0=
4,4,++QYBUGIUXrSsYfBPDW4PN5V6wnlJzLYCwP+FS9gMnY=


In [10]:
#On garde uniquement les chansons des 7678 utilisateurs
usr_list = list(usr_most.msno)
data = pd.DataFrame()
for usr in usr_list:
    data = data.append(trainPart[trainPart.msno == usr])


In [11]:
#Nombre de données restantes
len(data)

282251

In [12]:
data.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target
65094,+/SKX44s4ryWQzYzuV7ZKMXqIKQMN1cPz3M8CJ8CFKU=,SiEK8IZqD9pbPgaG2GZWVO/WKE1e7Hnqhn/vlML3cqE=,my library,Local playlist more,local-library,1
65095,+/SKX44s4ryWQzYzuV7ZKMXqIKQMN1cPz3M8CJ8CFKU=,YN4T/yvvXtYrBVN8KTnieiQohHL3T9fnzUkbLWcgLro=,discover,Online playlist more,online-playlist,1
65096,+/SKX44s4ryWQzYzuV7ZKMXqIKQMN1cPz3M8CJ8CFKU=,9kvPtsVoRFyh96q8Sk+UiMAPPTaeVvwUAjAUXgpj9NM=,my library,Local playlist more,local-library,1
71256,+/SKX44s4ryWQzYzuV7ZKMXqIKQMN1cPz3M8CJ8CFKU=,KZ5hwP74wRO6kRapVIprwodtNdVD2EVD3hkZmmyXFPk=,discover,Online playlist more,online-playlist,1
71257,+/SKX44s4ryWQzYzuV7ZKMXqIKQMN1cPz3M8CJ8CFKU=,3VkD5ekIf5duJm1hmYTZlXjyl0zqV8wCzuAh3uocfCg=,discover,Online playlist more,online-playlist,1


In [13]:
#Encodage des variables catégorielles
encode_lab = LabelEncoder()
dataEncode = pd.get_dummies(data, columns=['source_system_tab', 'source_screen_name', 'source_type'])

In [14]:
dataEncode.columns

Index(['msno', 'song_id', 'target', 'source_system_tab_discover',
       'source_system_tab_explore', 'source_system_tab_listen with',
       'source_system_tab_my library', 'source_system_tab_notification',
       'source_system_tab_null', 'source_system_tab_radio',
       'source_system_tab_search', 'source_system_tab_settings',
       'source_screen_name_Album more', 'source_screen_name_Artist more',
       'source_screen_name_Discover Chart',
       'source_screen_name_Discover Feature',
       'source_screen_name_Discover Genre', 'source_screen_name_Discover New',
       'source_screen_name_Explore', 'source_screen_name_Local playlist more',
       'source_screen_name_My library', 'source_screen_name_My library_Search',
       'source_screen_name_Online playlist more',
       'source_screen_name_Others profile more', 'source_screen_name_Radio',
       'source_screen_name_Search', 'source_screen_name_Search Home',
       'source_screen_name_Search Trends',
       'source_screen_nam

In [15]:
# On modifie les colonnes de chansons et membres pour avoir que des valeurs numériques
dataEncode = dataEncode.merge(song_id, left_on='song_id', right_on='song_id')
dataEncode = dataEncode.merge(usr_id, left_on='msno', right_on='msno')

In [16]:
dataEncode = dataEncode.iloc[:, 2:]

In [23]:
X = dataEncode.iloc[:, 1:]
Y = dataEncode['target']

In [24]:
#Split en train test
Xav, Xt, Yav, Yt = train_test_split(X, Y, shuffle=True, test_size=0.2, stratify=Y)
Xa, Xv, Ya, Yv = train_test_split(Xav, Yav, shuffle=True, test_size=0.3, stratify=Yav)

In [25]:
#%% Normalisation
sc = StandardScaler(with_mean=True, with_std=True)
sc = sc.fit(Xav)
Xav = sc.transform(Xav)
Xa = sc.transform(Xa)
Xv = sc.transform(Xv)
Xt = sc.transform(Xt)

In [ ]:
#%% Définition du modèle
paramC = 1
paramKer = 1
#Les paramètres de C sont les paramètres de base et seront modifiés par la suite
clf_ker = SVC(kernel='rbf', C = paramC, gamma=paramKer)
# apprentissage des parametres du SVM non Lineaire sur le jeu d'apprentissage
clf_ker.fit(Xa, Ya)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [ ]:
#%% Grid search (exploration des plages de valeurs de C et gamma)
vectC = np.logspace(-3, 2, 6)
vectGamma = np.logspace(-2, 2, 6)
err_val = np.empty((vectC.shape[0], vectGamma.shape[0]))
err_app = np.empty((vectC.shape[0], vectGamma.shape[0])) 
for ind_C, C in enumerate(vectC):
    print("- Apprentissage du SVM pour C = {:6.3f} ".format(C)) 
    clf_ker.C = C
    for ind_gam, paramKer in enumerate(vectGamma):
        print("\t Apprentissage du SVM pour gamma = {:6.2f} ".format( paramKer))
        clf_ker.gamma = paramKer
        clf_ker.fit(Xa, Ya)
        err_val[ind_C, ind_gam] = 1 - accuracy_score(Yv, clf_ker.predict(Xv))
        err_app[ind_C, ind_gam] = 1 - accuracy_score(Ya, clf_ker.predict(Xa))

plt.subplot(121)
plt.imshow(err_val, aspect="auto")
plt.title("Err validation")
plt.colorbar()
plt.show()
plt.subplot(122)
plt.imshow(err_app, aspect="auto")
plt.title("Err apprentissage")
plt.colorbar()
plt.show()

- Apprentissage du SVM pour C =  0.001 
	 Apprentissage du SVM pour gamma =   0.01 
	 Apprentissage du SVM pour gamma =   0.06 
	 Apprentissage du SVM pour gamma =   0.40 
	 Apprentissage du SVM pour gamma =   2.51 
	 Apprentissage du SVM pour gamma =  15.85 
	 Apprentissage du SVM pour gamma = 100.00 
- Apprentissage du SVM pour C =  0.010 
	 Apprentissage du SVM pour gamma =   0.01 
	 Apprentissage du SVM pour gamma =   0.06 
	 Apprentissage du SVM pour gamma =   0.40 


In [ ]:
# Apprentissage modele optimal et evaluation
clf_ker.C = Copt
clf_ker.gamma = GammaOpt
clf_ker.fit(Xav, Yav)
err_app = 1 - accuracy_score(Yav, clf_ker.predict(Xav)) 
print("\nSVM kernel optimal : erreur apprentissage = {}%".format(100*err_app))

err_test = 1 - accuracy_score(Yt, clf_ker.predict(Xt))
print("SVM kernel optimal : erreur test = {}%".format(100*err_test))

In [26]:
#%% Régression logistique
#On va d'abord déterminer la valeur optimale du paramètre C à utiliser.

clf_reglog = linear_model.LogisticRegression(tol=1e-5, multi_class='multinomial', solver='lbfgs')
#Les valeurs de C que je teste sont entre 10^-3 et 10
# Les tests initiaux avec d'autres valeurs ont montré qu'au dela de ces limites, l'erreur est stable
vectC = np.logspace(-3, 1, 20)
err_app = np.empty(vectC.shape[0])
err_val = np.empty(vectC.shape[0])
for ind_C, C in enumerate(vectC):
    clf_reglog.C = C
    clf_reglog.fit(Xa, Ya)
    err_val[ind_C] = 1 - accuracy_score(Yv, clf_reglog.predict(Xv))
    err_app[ind_C] = 1 - accuracy_score(Ya, clf_reglog.predict(Xa))


In [27]:
# Choix du meilleur C
err_min_val, ind_min = err_val.min(), err_val.argmin()
Copt = vectC[ind_min]

In [28]:
#%% On utilise ensuite la meilleure valeur de C sur les données de tests pour voir si la prédiction est toujours performante
clf_reglog.C = Copt
clf_reglog.fit(Xav, Yav)
print('Valeur de Copt = {}'.format(Copt ))
print('RegLog: Err test correspondante = {}'.format(100*(1 - accuracy_score(Yt, clf_reglog.predict(Xt)))))
print('RegLog: Err apprentissage correspondante = {}'.format(100*(1 - accuracy_score(Yav, clf_reglog.predict(Xav)))))

Valeur de Copt = 10.0
RegLog: Err test correspondante = 23.671857008733234
RegLog: Err apprentissage correspondante = 23.64083259521701
